# Assignment 5 - Issue #854

Fernando Mendoza | Adrea Pezo | Michel Cotrina | Estefanny Gil | Armando Ore

The script should not give an error. Any mistake will be consider as 0.

1. We want an excel file with all the available presidential elections in this webpage.
https://infogob.jne.gob.pe/Eleccion

2. Go to Candidatos y Resultados

In [ ]:
# For Chrome only
#from selenium import webdriver
#from selenium.webdriver.chrome.service import Service as ChromeService
#from webdriver_manager.chrome import ChromeDriverManager

#from selenium.webdriver.support.ui import Select
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.by import By
#import re
#import time 
#import pandas as pd

# Path for Chrome executable
#chrome_path = r'chrome-win64\chrome.exe'

# Configure the options for Chrome
#options = webdriver.ChromeOptions()
#options.binary_location = chrome_path

# Configure the service for Chrome
#service = ChromeService(executable_path=ChromeDriverManager().install())

# Start the WebDriver for Chrome
#driver = webdriver.Chrome(service=service, options=options)


In [119]:
#For FireFox only
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import re
import time 
import pandas as pd

# Path for geckodriver (Firefox)
gecko_driver_path = r'geckodriver-v0.34.0-win64\geckodriver.exe'

# Configure the service for Firefox
service = FirefoxService(executable_path=gecko_driver_path)

# Start the WebDriver for Firefox
driver = webdriver.Firefox(service=service)


In [120]:
# Maximize window
driver.maximize_window()

# Base URL for the web scraping. Open a new window
url_base = "https://infogob.jne.gob.pe/Eleccion"
driver.get(url_base)


In [121]:
# Display "Tipo de proceso" options
opt_Presidenciales = driver.find_element(By.XPATH,'/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[1]')
opt_Presidenciales.click()

# Select "ELECCIONES PRESIDENCIALES"
opt_Presidenciales_2 = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[2]/div[2]/div[2]/div[1]/div/div[2]/div[2]')
opt_Presidenciales_2.click()

In [127]:
#Getting the number of options
#elections_0 = driver.find_element(By.XPATH, "/html/body/div[1]/section/div[2]/div[2]/div[2]/div[2]/div/select" )
#election = Select(elections_0)
#num_election_options = len( election.options )
#num_election_options 

elections_0 = driver.find_element(By.XPATH, '//*[@id="IdEleccion"]')
election = Select(elections_0)
num_election_options = len( election.options )
num_election_options 

26

In [129]:
options = election.options
for option in options:
    print(option.text)

In [131]:
#Getting the name of the elections
for election_idx in range( num_election_options ):
    
    election = Select(elections_0)
    election_opt = election.options[ election_idx ]
    
    election_name = election_opt.text
print(election_name)

3. Go to Candidatos y Resultados

4. Get this table

https://private-user-images.githubusercontent.com/69997588/296330169-20de4ccb-d61a-40a2-b9b3-de17ef4d5782.png?jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmF3LmdpdGh1YnVzZXJjb250ZW50LmNvbSIsImtleSI6ImtleTUiLCJleHAiOjE3MjQyODY5NDMsIm5iZiI6MTcyNDI4NjY0MywicGF0aCI6Ii82OTk5NzU4OC8yOTYzMzAxNjktMjBkZTRjY2ItZDYxYS00MGEyLWI5YjMtZGUxN2VmNGQ1NzgyLnBuZz9YLUFtei1BbGdvcml0aG09QVdTNC1ITUFDLVNIQTI1NiZYLUFtei1DcmVkZW50aWFsPUFLSUFWQ09EWUxTQTUzUFFLNFpBJTJGMjAyNDA4MjIlMkZ1cy1lYXN0LTElMkZzMyUyRmF3czRfcmVxdWVzdCZYLUFtei1EYXRlPTIwMjQwODIyVDAwMzA0M1omWC1BbXotRXhwaXJlcz0zMDAmWC1BbXotU2lnbmF0dXJlPTQ3OTE4OTJkNzZkNjI4Mjk4NGJiZDdjZGMxN2U0MmYzYTY4M2Q4ODQ4Yzc5Zjg5Mzk2MmFmZWQyNGQwN2E1YmYmWC1BbXotU2lnbmVkSGVhZGVycz1ob3N0JmFjdG9yX2lkPTAma2V5X2lkPTAmcmVwb19pZD0wIn0.ErRsdd3ug4VjnRodkLIY8Cz61xgnbuYIobPx_fo-UQY

5.  Do the same for all the options available here

6. The final excel should have these 3 columns 
(Elecciones | ORGANIZACIÓN POLÍTICA | TOTAL VOTOS)